In [83]:
import xml.etree.cElementTree as ET
import random
import json
import sys
import torch.nn.functional as F

In [84]:
train_target = ET.parse('/Users/a.sychev/Desktop/russian-bert-absa-api/lib/data/sentence_level/train/train.xml')
train_data = {'annotation':[]}

test_target = ET.parse('/Users/a.sychev/Desktop/russian-bert-absa-api/lib/data/sentence_level/test.xml')
test_data = {'annotation':[]}

In [85]:
for obj in train_target.iter('Review'):
    sent = obj.iter('sentence')
    for se in sent:
        text = se.find('text').text.strip()
        cat = []
        pol = []
        for c in se.iter('Opinion'):
            cat.append(c.attrib['category'])
            pol.append(c.attrib['polarity'])
        if cat!=[]:
            train_data['annotation'].append({'text':text, 'category':cat, 'polarity': pol})

for obj in test_target.iter('Review'):
    sent = obj.iter('sentence')
    for se in sent:
        text = se.find('text').text.strip()
        cat = []
        pol = []
        for c in se.iter('Opinion'):
            cat.append(c.attrib['category'])
            pol.append(c.attrib['polarity'])
        if cat!=[]:
            test_data['annotation'].append({'text':text, 'category':cat, 'polarity': pol})

In [86]:
test_data['annotation'][2]

{'text': 'После того как персонал сменился,мне не раз путали блюда,которые я заказывалаьи приносили совсем другое,что-то и вовсе забывали.',
 'category': ['SERVICE#GENERAL'],
 'polarity': ['negative']}

In [87]:
test_data['annotation']

[{'text': 'Безумно люблю этот ресторан!',
  'category': ['RESTAURANT#GENERAL'],
  'polarity': ['positive']},
 {'text': 'Прекрасная атмосфера и безумно вкусные блюда!',
  'category': ['AMBIENCE#GENERAL', 'FOOD#QUALITY'],
  'polarity': ['positive', 'positive']},
 {'text': 'После того как персонал сменился,мне не раз путали блюда,которые я заказывалаьи приносили совсем другое,что-то и вовсе забывали.',
  'category': ['SERVICE#GENERAL'],
  'polarity': ['negative']},
 {'text': 'Ребёнок как-то разлил коктейль,так мы минут 5 не могли дозваться официанта и сами вытирали чем могли!',
  'category': ['SERVICE#GENERAL'],
  'polarity': ['negative']},
 {'text': 'А о том,что они мало что знают о своих блюдах,я уже и не говорю!',
  'category': ['SERVICE#GENERAL'],
  'polarity': ['negative']},
 {'text': 'Еда осталась такой же вкусной...',
  'category': ['FOOD#QUALITY'],
  'polarity': ['positive']},
 {'text': 'Но сейчас такого огромного желания нет.',
  'category': ['RESTAURANT#GENERAL'],
  'polarity': 

In [88]:
categories = []
for anno in train_data['annotation']:
    for cat in anno['category']:
        categories.append(cat)

for anno in test_data['annotation']:
    for cat in anno['category']:
        categories.append(cat)

In [89]:
train_data_len = [i for i in range(len(train_data['annotation']))]
len(train_data['annotation'])

3408

In [90]:
test_data_len = [i for i in range(len(test_data['annotation']))]
len(test_data['annotation'])

300

In [91]:
categories = []
for anno in train_data['annotation']:
    for cat in anno['category']:
        categories.append(cat)
set(categories)

{'AMBIENCE#GENERAL',
 'FOOD#QUALITY',
 'FOOD#STYLE_OPTIONS',
 'RESTAURANT#GENERAL',
 'RESTAURANT#PRICES',
 'SERVICE#GENERAL'}

In [92]:
categories = []
for anno in test_data['annotation']:
    for cat in anno['category']:
        categories.append(cat)
len(set(categories))

6

In [93]:
def get_duplicates_indexes(collection, item):
    return [i for i, x in enumerate(collection) if x == item]

In [94]:
for review in train_data['annotation']:
    for review_category in review['category']:
        indexes_of_duplicates_items = get_duplicates_indexes(review['category'], review_category)
        if (len(indexes_of_duplicates_items) > 1):
            result_set = set()
            for index_of_duplicate_item in indexes_of_duplicates_items:
                result_set.add(review['polarity'][index_of_duplicate_item]) 
            if (len(result_set) > 1):
                print(result_set, review['text'])

In [95]:
for review in test_data['annotation']:
    for review_category in review['category']:
        indexes_of_duplicates_items = get_duplicates_indexes(review['category'], review_category)
        if (len(indexes_of_duplicates_items) > 1):
            result_set = set()
            for index_of_duplicate_item in indexes_of_duplicates_items:
                result_set.add(review['polarity'][index_of_duplicate_item]) 
            if (len(result_set) > 1):
                print(result_set, review['text'])

In [96]:
with open("train.json", "w", encoding='utf-8') as fp:
    json.dump(train_data, fp, ensure_ascii=False) 
with open("test.json", "w", encoding='utf-8') as fp:
    json.dump(test_data, fp, ensure_ascii=False) 

In [101]:
with open("Otzyvy.json", "r", encoding='utf-8') as file:
    generated_data = json.load(file)

with open("train.json", "r+", encoding='utf-8') as file:
    file_data = json.load(file)
    file_data['annotation'].append(generated_data['annotation'])
    file.seek(0)
    json.dump(file_data, file, ensure_ascii=False)